# Artwork Sequence Prediction Evaluation

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'

In [4]:
window_index = 3

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

{'weights_folder': '/root/work/artwork_sequence/train_test_configuration/univariate/config_3/trained_model_weights',
 'x_test': '/root/work/artwork_sequence/train_test_configuration/univariate/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/train_test_configuration/univariate/X_test_matrix.npy',
 'x_train': '/root/work/artwork_sequence/train_test_configuration/univariate/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/train_test_configuration/univariate/X_train_matrix.npy'}

## Load data

In [5]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

,tour_path
20,/root/work/datasets/artwork_sequence/rijksmuse...
7,/root/work/datasets/artwork_sequence/rijksmuse...
40,/root/work/datasets/artwork_sequence/prado_cra...
0,/root/work/datasets/artwork_sequence/rijksmuse...
23,/root/work/datasets/artwork_sequence/prado_cra...


In [6]:
x_train_matrix.shape

(637, 300)

In [7]:
df_x_test['tour_path'][39]

'/root/work/datasets/artwork_sequence/prado_crawler/tour_18'

## Reset Tensorflow session

In [8]:
tf.keras.backend.clear_session()

## Config  data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [9]:
timeline = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
timeline.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [10]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

window_size = window_index

#Number of artwork's feature
n_features = X.shape[1]

#Number of feature to take into account
n_influence_features=3

batch_size = 128
shuffle_buffer_size = 300

## Create and Load models

**Get models**

In [ ]:
from Prediction_model_feature import Prediction_model_feature
from Prediction_model_feature_multivariate import Prediction_model_feature_multivariate

In [ ]:
models = []

In [ ]:
from IPython.display import clear_output
import time

start_time = time.time()

for i in range(n_features):
    clear_output(wait=True)
    #Create model
    model_prediction = Prediction_model_feature_multivariate(
        X=X,
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(i),
        index = i,
        n_influence_features=n_influence_features,
        n_features=1)
    model_prediction.define_model(conv_filter=32, lstm_filter=64, dense_filter=16)

    #Load weights
    model_prediction.load_weights(museum_sequence_path)

    models.append(model_prediction)
    
print("--- %s seconds ---" % (time.time() - start_time))
len(models)

## Evaluate model

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    if len(series.shape) == 1:
            series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.map(lambda w: (w[:]))
    ds = ds.batch(batch_size)
    forecast = model.predict(ds)
    return forecast

**Predict validation dataset**

In [ ]:
df_evaluation = pd.DataFrame({'feature' : [],
                             'forecast': [],
                             'x_valid':[],
                             'mae':[]})
df_evaluation

In [ ]:
for feature in range(n_features):
    
    #Define feature to take into account for prediction
    x_influence_features = models[feature].get_indexes_features()
    x_influence_features = np.insert(arr=x_influence_features, obj=0, values=feature)
    x_feature = X[:,x_influence_features]
    #Predict feature i
    rnn_forecast = model_forecast(models[feature].get_model(), x_feature, window_size, batch_size)
    rnn_forecast = rnn_forecast[split_time-window_size+1:,-1]

    #Get validation dataset 
    x_valid = x_feature[split_time:, 0]

    #Compute MAE
    mae = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy().mean()

    df_evaluation = df_evaluation.append({'feature' : feature,
                    'forecast': rnn_forecast,
                    'x_valid':x_valid,
                    'mae':mae
                   }, 
                   ignore_index=True)

df_evaluation.head()

In [ ]:
df_evaluation.to_csv(os.path.join(CONFIG_PATH, 'config_'+str(window_size)+'/forecast.csv'), index=False)

**Plot forecast for a single feature**

In [ ]:
PLOT_PATH = '/root/work/artwork_sequence/plots'

In [ ]:
feature = 5
rnn_forecast = df_evaluation.loc[feature,'forecast']
x_valid = df_evaluation.loc[feature,'x_valid']
timeline = np.arange(rnn_forecast.shape[0])

In [ ]:
plot = plot_series(timeline, [(x_valid, 'x_valid'), (rnn_forecast, 'rnn')])
plot.title('Forescast artwork sequence')
plot.savefig(os.path.join(PLOT_PATH, 'forescast_sequence_2.png'))
plot.show()

**MAE**

In [ ]:
df_evaluation['mae'].mean()